### Preparação do texto

In [1]:
import pdfplumber
import os
import torch
from langchain import LLMChain, PromptTemplate
from langchain.llms import HuggingFaceHub
from langchain.retrievers import TFIDFRetriever
from dotenv import load_dotenv
import pandas as pd
from langchain import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

In [2]:
load_dotenv()
# Acesse as variáveis de ambiente
path_pdfs = os.path.abspath(os.path.join(os.getcwd(), '..', '..', os.getenv('DIRECTORY_PDFS')))
token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
import pandas as pd

# Função para extrair texto de PDFs e separar em seções
def extract_text_from_pdfs(directory):
    registros = []
    for file in os.listdir(directory):
        if file.endswith('.pdf'):
            file_path = os.path.join(directory, file)
            with pdfplumber.open(file_path) as pdf:
                text = ''
                for page in pdf.pages:
                    text += page.extract_text()
                # Encontrar os índices dos registros
                indexes = [i for i in range(len(text)) if text.startswith("REGISTRO", i)]
                # Extrair as seções
                for i in range(len(indexes)):
                    start = indexes[i]
                    if i == len(indexes) - 1:
                        end = len(text)
                    else:
                        end = indexes[i + 1]
                    registro_text = text[start:end].strip()
                    # Extrair o identificador do registro
                    registro_id = registro_text.split(":")[0].split()[-1]
                    # Adicionar linha ao DataFrame
                    registros.append({"registro_id": registro_id, "registro_text": registro_text})
    # Criar o DataFrame
    df = pd.DataFrame(registros)
    
    # Calcular o tamanho médio de cada seção
    df['tamanho'] = df['registro_text'].apply(len)
    tamanho_medio = df['tamanho'].mean()
    print(f"Tamanho médio de cada seção: {tamanho_medio} caracteres")
    
    # Calcular o tamanho médio de cada seção em palavras
    df['tamanho_palavras'] = df['registro_text'].apply(lambda x: len(x.split()))
    tamanho_medio_palavras = df['tamanho_palavras'].mean()
    print(f"Tamanho médio de cada seção: {tamanho_medio_palavras} palavras")
    
    return df

In [4]:
# Extraia o texto dos PDFs
registros = extract_text_from_pdfs(path_pdfs)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Tamanho médio de cada seção: 7054.555555555556 caracteres
Tamanho médio de cada seção: 1146.2222222222222 palavras


In [5]:
print(registros)

  registro_id                                      registro_text  tamanho  \
0        0190  REGISTRO 0190: IDENTIFICAÇÃO DAS UNIDADES DE M...      716   
1        0200  REGISTRO 0200: TABELA DE IDENTIFICAÇÃO DO ITEM...    12057   
2        0205  REGISTRO 0205: ALTERAÇÃO DO ITEM\nEste registr...     1350   
3        C001  REGISTRO C001: ABERTURA DO BLOCO C\nEste regis...      768   
4        C100  REGISTRO C100: NOTA FISCAL (CÓDIGO 01), NOTA F...    24516   
5        C101  REGISTRO C101: INFORMAÇÃO COMPLEMENTAR DOS DOC...      783   
6        C165  REGISTRO C165: OPERAÇÕES COM COMBUSTÍVEIS (CÓD...     1803   
7        C170  REGISTRO C170: ITENS DO DOCUMENTO (CÓDIGO 01, ...    20734   
8        C171  REGISTRO C171: ARMAZENAMENTO DE COMBUSTÍVEIS (...      764   

   tamanho_palavras  
0               123  
1              1906  
2               227  
3               134  
4              4003  
5               129  
6               309  
7              3361  
8               124  


### Embeddings, Vector, Retriever

In [6]:
# Criar um índice de embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
docs = []
for index, row in registros.iterrows():
    doc = Document(page_content=row["registro_text"], metadata={"registro_id": row["registro_id"]})
    docs.append(doc)


C:\Users\clr_c\AppData\Local\Temp\ipykernel_14720\3539359712.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\clr_c\anaconda3\envs\efd-ia\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Criar um Vetor Store com FAISS
vectorstore = FAISS.from_documents(docs, embeddings)

In [8]:
# Criar um retriever
retriever = vectorstore.as_retriever()


### Criação do Modelo RAG de Texto Bruto com Modelo do HaggindFace

In [9]:
# Criar um modelo de linguagem

llm = HuggingFaceEndpoint(
    endpoint_url="https://api-inference.huggingface.co/models/google/flan-t5-small",
    huggingfacehub_api_token=token,
    temperature=0.7,
    top_p=0.95,
    model_kwargs={
        "max_new_tokens ": 250
    }
)

In [10]:
# Criar uma cadeia de QA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

### Criação do Modelo RAG de Texto Bruto com Modelo Local usando Biblioteca Transformers

In [13]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
from langchain_huggingface import HuggingFacePipeline
import torch

In [14]:
# Carregar o tokenizador e o modelo localmente
model_name = "google/flan-t5-small"
print(f"Carregando modelo {model_name} localmente...")
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


Carregando modelo google/flan-t5-small localmente...


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
# Caso contrário, usará a CPU (device=-1)
device = 0 if torch.cuda.is_available() else -1
print(f"Usando dispositivo: {'GPU' if device == 0 else 'CPU'}")

Usando dispositivo: GPU


In [16]:

# Criar o pipeline do transformers
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250, # Definir o limite de tokens aqui
    temperature=0.7,
    top_p=0.95,
    device=device # Especificar o dispositivo
)


Device set to use cuda:0


In [17]:
# Criar o LLM para LangChain a partir do pipeline
llm = HuggingFacePipeline(pipeline=pipe)

print("Modelo FLAN-T5-small carregado e pipeline criado.")

Modelo FLAN-T5-small carregado e pipeline criado.


In [18]:
# Criar uma cadeia de QA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

### Teste do RAG

In [11]:
# Perguntas
#pergunta = "Qual é o registro mais relevante?"
pergunta = "Em quais registros o campo COD_ITEM é usado?"

In [19]:
# Testar o sistema de RAG

resultado = qa_chain.invoke({"query": pergunta})
print("Resposta:", resultado["result"])
print("Documentos fonte:")
for doc in resultado["source_documents"]:
    print(doc.page_content)

Token indices sequence length is longer than the specified maximum sequence length for this model (7322 > 512). Running this sequence through the model will result in indexing errors
c:\Users\clr_c\anaconda3\envs\efd-ia\lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\clr_c\anaconda3\envs\efd-ia\lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Resposta: OC 02 Registro 0150
Documentos fonte:
REGISTRO 0190: IDENTIFICAÇÃO DAS UNIDADES DE MEDIDA
Este registro tem por objetivo descrever as unidades de medidas utilizadas no arquivo digital. Não podem ser
informados dois ou mais registros com o mesmo código de unidade de medida. Somente devem constar as unidades de medidas
informadas em qualquer outro registro.
Nº Campo Descrição Tipo Tam Dec Obrig
01 REG Texto fixo contendo "0190" C 004 - O
02 UNID Código da unidade de medida C 006 - O
03 DESCR Descrição da unidade de medida C - - O
Observações:
Nível hierárquico: 2
Ocorrência: vários por arquivo
Campo 01 (REG) - Valor Válido: [0190]
Campo 02 (UNID) - Validação: o valor informado neste campo deve existir em, pelo menos, um outro registro do arquivo.
REGISTRO 0205: ALTERAÇÃO DO ITEM
Este registro tem por objetivo informar alterações ocorridas na descrição do produto ou quando ocorrer alteração na
codificação do produto, desde que não o descaracterize ou haja modificação que o ident